In [6]:
!pip install pandas
!pip install gensim
!pip install pyLDAvis
!pip install PyMySQL
!pip install tqdm

    100% |████████████████████████████████| 51kB 3.5MB/s 


In [2]:
|!git clone https://github.com/snowballstem/snowball

Cloning into 'snowball'...
remote: Counting objects: 3399, done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 3399 (delta 84), reused 121 (delta 83), pack-reused 3267
Receiving objects: 100% (3399/3399), 911.03 KiB | 7.99 MiB/s, done.
Resolving deltas: 100% (2159/2159), done.


In [0]:
import os
import datetime
import pymysql
import time
import pandas as pd
import numpy as np
import string
import subprocess
import urllib
from tqdm import tqdm


In [12]:
!ls

algorithms     COPYING	    go		libstemmer    runtime
algorithms.mk  csharp	    iconv.py	libstemmer.o  rust
AUTHORS        doc	    include	pascal	      snowball
charsets       examples     java	python	      src_c
compiler       GNUmakefile  javascript	README	      stemwords


In [0]:
os.chdir('snowball')

In [11]:
!make

cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude  -c -o compiler/space.o compiler/space.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude  -c -o compiler/tokeniser.o compiler/tokeniser.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude  -c -o compiler/analyser.o compiler/analyser.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude  -c -o compiler/generator.o compiler/generator.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude  -c -o compiler/driver.o compiler/driver.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude  -c -o compiler/generator_csharp.o compiler/generator_csharp.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude  -c -o compiler/generator_java.o compiler/generator_java.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude  -c -o compiler/generator_js.o compiler/generator_js.c
cc -O2 -W -Wall -Wmissing-prototyp

cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude -c -o src_c/stem_UTF_8_arabic.o src_c/stem_UTF_8_arabic.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude -c -o src_c/stem_UTF_8_danish.o src_c/stem_UTF_8_danish.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude -c -o src_c/stem_UTF_8_dutch.o src_c/stem_UTF_8_dutch.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude -c -o src_c/stem_UTF_8_english.o src_c/stem_UTF_8_english.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude -c -o src_c/stem_UTF_8_finnish.o src_c/stem_UTF_8_finnish.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude -c -o src_c/stem_UTF_8_french.o src_c/stem_UTF_8_french.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude -c -o src_c/stem_UTF_8_german.o src_c/stem_UTF_8_german.c
cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude -c -o src_c/stem_UTF_8_hungarian

cc -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations -Iinclude -c -o src_c/stem_ISO_8859_2_romanian.o src_c/stem_ISO_8859_2_romanian.c
ar -cru libstemmer.o libstemmer/libstemmer.o runtime/api.o runtime/utilities.o src_c/stem_UTF_8_arabic.o src_c/stem_UTF_8_danish.o src_c/stem_UTF_8_dutch.o src_c/stem_UTF_8_english.o src_c/stem_UTF_8_finnish.o src_c/stem_UTF_8_french.o src_c/stem_UTF_8_german.o src_c/stem_UTF_8_hungarian.o src_c/stem_UTF_8_indonesian.o src_c/stem_UTF_8_irish.o src_c/stem_UTF_8_italian.o src_c/stem_UTF_8_lithuanian.o src_c/stem_UTF_8_nepali.o src_c/stem_UTF_8_norwegian.o src_c/stem_UTF_8_porter.o src_c/stem_UTF_8_portuguese.o src_c/stem_UTF_8_romanian.o src_c/stem_UTF_8_russian.o src_c/stem_UTF_8_spanish.o src_c/stem_UTF_8_swedish.o src_c/stem_UTF_8_tamil.o src_c/stem_UTF_8_turkish.o src_c/stem_KOI8_R_russian.o src_c/stem_ISO_8859_1_danish.o src_c/stem_ISO_8859_1_dutch.o src_c/stem_ISO_8859_1_english.o src_c/stem_ISO_8859_1_finnish.o src_c/stem_ISO_8859_1_french.o

In [0]:
def join_name_description(row):
    try:
        return ' '.join([row[4],row[2]])
    except:
        return np.nan
      
def full_text_processing(full_text):
    try:
        words = full_text.split()
    except:
        return np.nan
    # remove punctuation from each word
    table = str.maketrans('','', string.punctuation)
    table2 = str.maketrans('', '', string.digits)
    stripped = [w.translate(table) for w in words]
    stripped2 = [w.translate(table2) for w in stripped]
    stripped3 = [w.replace('„','').replace('”','').replace('“','') for w in stripped2 if w != '–']
    text_only_words = [x for x in stripped3 if x]
    cleaned_text = [y for y in text_only_words if y not in stop_words]
    thefile = open('test_lt.txt', 'w')
    for item in cleaned_text:
        thefile.write("%s\n" % item)
    thefile.close()
    args = ("./stemwords", "-l", "lt", "-i", "test_lt.txt", "-o", "out_lt.txt")
    popen = subprocess.Popen(args, stdout=subprocess.PIPE)
    popen.wait()
    with open('out_lt.txt') as f:
        lst = f.readlines()
        lst_cl = [x.rstrip() for x in lst]
    f.close()
    return cleaned_text, lst_cl
  
def using_stemmed_dictionary(full_text, dic):
    words = full_text.split()
    table = str.maketrans('','', string.punctuation)
    table2 = str.maketrans('', '', string.digits)
    stripped = [w.translate(table) for w in words]
    stripped2 = [w.translate(table2) for w in stripped]
    stripped3 = [w.replace('„','').replace('”','').replace('“','') for w in stripped2 if w != '–']
    text_only_words = [x for x in stripped3 if x]
    cleaned_text = [y for y in text_only_words if y not in stop_words]
    return [dic[x] for x in cleaned_text]

In [0]:
data = urllib.request.urlopen('https://gist.githubusercontent.com/revelt/01524e76c6e5e0970d2d0fe8797e92ed/raw/48258f1b1055f62d4ef7ed51fc0411aa94358091/Lithuanian%2520stop%2520words')# it's a file like object and works just like a file
stop_words = [line.decode('utf-8')[:-1] for line in data]
stop_words.append('iš')
#knygu aprasymuose daznai minimas zodis knyga neteikia konsteksto
stop_words.append('knyga')

In [0]:
tqdm.pandas()

In [0]:
#item_frame - padnas dataframe created from database with book title and description 


In [0]:
item_frame['full_text'] = item_frame.apply(join_name_description, axis = 1)
item_frame = item_frame[item_frame['full_text'].apply(lambda x: type(x)==str)]

In [0]:
books = ' '.join(item_frame.full_text.values)

In [0]:
original, cleaned = full_text_processing(books)
dictionary = dict(zip(original, cleaned))

In [19]:
item_frame['cleaned_text'] = item_frame.full_text.progress_apply(lambda x : using_stemmed_dictionary(x,dictionary))

100%|██████████| 62184/62184 [00:46<00:00, 1332.74it/s]
